### Import modules

In [1]:
import os
import pandas as pd

os.chdir("../../")

from datasets import load_dataset
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

from dotenv import load_dotenv

In [2]:
from src.rag_pipeline import chunk_by_recursive_split, RAGSystem
from src.env_loader import load_api_keys
from src.ragas.ragas_pipeline import run_ragas_evaluation
from src import display_df

### Load API keys

In [3]:
# openai_api_key = load_api_keys("OPENAI_API_KEY")
gemini_api_key = load_api_keys("GOOGLE_API_KEY")

In [4]:
llm_gemini = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85)

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

I0000 00:00:1722428028.471122 2298355 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


#### Initialize embeddings and RAG system

In [5]:
# embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

embeddings = gemini_embeddings

# embeddings_model = 'text-embedding-ada-002'
# embeddings_model = 'text-embedding-3-large'
# embeddings = OpenAIEmbeddings(api_key=openai_api_key, model=embeddings_model)

# embeddings=FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")


In [10]:
rag_system = RAGSystem(
  # model_name = "gpt-4o",
  existing_vectorstore = False,
  embeddings = embeddings,
  clear_store = True,
  llm = llm_gemini,
  # use_multiquery = True,
)

In [11]:
rag_system.initialize()

--SETUP NEW VECTORSTORE--
--Split 1000 documents into 5030 chunks.--
--USING BASE RETRIEVER--
--SETUP RAG CHAIN--
--RAGCHAIN SETUP COMPLETE!--


#### Test the RAG Chain

In [16]:
question = "Who was Putin?"
result = rag_system.rag_chain.invoke(question)

In [17]:
result

{'question': 'Who was Putin?',
 'answer': 'I do not have enough information to answer this question.',
 'contexts': ['16, 2014, Crimea held a referendum on whether to continue as part of Ukraine or to cast its lot with the Russian Federation. The result was overwhelming, at least among those voting: 95% said they wanted to become part of Russia. A mere two days later, Russia annexed the territory. Putin said later that he had been planning the move even before the referendum was held. "Friends!" the President said in Red Square on Wednesday, according to his website, "Exactly one year ago, Russia, which we are speaking of so much right now, and the Russian people showed amazing togetherness and patriotism in supporting the aspirations of the people of Crimea and Sevastopol to return to their native shores." The issue, he said, was "our history, our spirituality and our statehood, the things that make us a single people and single united nation." Also participating, the website said, we

## RAGAS Pipeline testing the rag_chain

### Ragas Testing with Langsmith Tracing

In [7]:
# experiment_name = "baseline_rag_benchmark_1"
# dataset_name = "cnn_dailymail_evaluation"

# rag_results = run_ragas_evaluation(
#   rag_chain=rag_system.rag_chain,
#   use_langsmith=True,
#   experiment_name=experiment_name,
#   dataset_name=dataset_name,
#   upload_dataset_to_langsmith=True,
#   save_results=True
# )

--LOADING EVALUATION DATA--
--GETTING CONTEXT AND ANSWERS--
--USING LANGSMITH FOR EVALUATION--
Created a new dataset 'cnn_dailymail_evaluation'. Dataset is accessible at https://smith.langchain.com/o/6691a6dd-a70e-56c0-8f45-a1f64338d797/datasets/8e291ee7-635e-40c2-ab54-1d2e8897e5f6
View the evaluation results for project 'baseline_rag_benchmark' at:
https://smith.langchain.com/o/6691a6dd-a70e-56c0-8f45-a1f64338d797/datasets/8e291ee7-635e-40c2-ab54-1d2e8897e5f6/compare?selectedSessions=a58cdd46-9bf6-44ae-9ea4-f0853631205f

View all tests for Dataset cnn_dailymail_evaluation at:
https://smith.langchain.com/o/6691a6dd-a70e-56c0-8f45-a1f64338d797/datasets/8e291ee7-635e-40c2-ab54-1d2e8897e5f6
[------------>                                     ] 5/19

Error evaluating run f591f3a5-4864-48c3-ac91-409ab305f428 with EvaluatorChain: APIConnectionError('Connection error.')
Traceback (most recent call last):
  File "/home/hilla/.cache/pypoetry/virtualenvs/rag-optimization-cnn-dailymail-hiPg4Kip-py3.10/lib/python3.10/site-packages/openai/_base_client.py", line 1558, in _request
    response = await self._client.send(
  File "/home/hilla/.cache/pypoetry/virtualenvs/rag-optimization-cnn-dailymail-hiPg4Kip-py3.10/lib/python3.10/site-packages/httpx/_client.py", line 1661, in send
    response = await self._send_handling_auth(
  File "/home/hilla/.cache/pypoetry/virtualenvs/rag-optimization-cnn-dailymail-hiPg4Kip-py3.10/lib/python3.10/site-packages/httpx/_client.py", line 1689, in _send_handling_auth
    response = await self._send_handling_redirects(
  File "/home/hilla/.cache/pypoetry/virtualenvs/rag-optimization-cnn-dailymail-hiPg4Kip-py3.10/lib/python3.10/site-packages/httpx/_client.py", line 1726, in _send_handling_redirects
    response =

[------------------------------------------------->] 19/19

,feedback.answer_correctness,feedback.faithfulness,feedback.answer_relevancy,feedback.context_precision,error,execution_time,run_id
count,19.000000,18.000000,18.000000,18.000000,0,19.000000,19
unique,NaN,NaN,NaN,NaN,0,NaN,19
top,NaN,NaN,NaN,NaN,NaN,NaN,31f949c4-1476-4eb2-ae11-f23eb62af6d3
freq,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,0.706439,0.851852,0.887768,0.965509,NaN,2.434766,NaN
std,0.203250,0.243470,0.225174,0.083576,NaN,0.693174,NaN
min,0.229624,0.250000,0.000000,0.679167,NaN,1.334236,NaN
25%,0.579877,0.687500,0.918437,1.000000,NaN,2.051280,NaN
50%,0.743723,1.000000,0.934425,1.000000,NaN,2.481985,NaN
75%,0.832633,1.000000,0.963321,1.000000,NaN,2.726066,NaN


--EVALUATION COMPLETE--


AttributeError: 'TestResult' object has no attribute 'to_pandas'

### Run Ragas tests locally

In [9]:
rag_results = run_ragas_evaluation(
  rag_chain=rag_system.rag_chain,
  save_results=True,
  experiment_name="embedding_model_bge_large"
)


--LOADING EVALUATION DATA--
--EVALUATING LOCALLY--
--GETTING CONTEXT AND ANSWERS--


Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

--EVALUATION COMPLETE--
--RESULTS SAVED--
